# Тетрадка с подготовкой данных <code>prepare_gis_houses_data.pickle</code>

<code>prepare_gis_houses_data.pickle</code> используется в <code>apartment_houses_all_data.ipynb</code> для создания и заливки в БД таблички <code>all_gis_houses_data</code>

Где взять сырые данные:
- нужно зайти по ссылке https://dom.gosuslugi.ru/#!/houses или найти поиском "Реестр объектов жилищного фонда"
- выбрать в Субъект РФ "Москва"
- нажать "Найти" внизу справа
- нажать "Скачать" вверху справа
- скачается папка/архив с несколькими csv, её и будем обрабатывать

In [1]:
import os
from typing import List, Optional
import pandas as pd
import numpy as np
from tqdm import tqdm

from postamats.utils import prepare_data
from postamats.utils import load
from postamats.utils.connections import DB
from postamats.utils.prepare_data import OBJECT_ID_COL, PREPARED_DATA_PATH, PREPARED_GIS_FILE, GEODATA_CENTER_COL

prepared_data_path = load.get_full_path_from_relative(PREPARED_DATA_PATH)
result_filepath = os.path.join(prepared_data_path, PREPARED_GIS_FILE)
print('result_filepath:', result_filepath)

pd.set_option('display.max_columns', None)
tqdm.pandas()
# https://dom.gosuslugi.ru/#!/houses

result_filepath: /Users/affernus/PROJECTS/hacks/postomat_optimisation/Notebooks/prepare_data/data/prepare_gis_houses_data.pickle


In [2]:
DATA_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022'

In [3]:
COLS_MAP = {
    'Адрес ОЖФ': 'ADDRESS_GIS',
    'Идентификационный код адреса дома в ГИС ЖКХ': 'ADDRESS_GIS_CODE',
    'Глобальный уникальный идентификатор дома по ФИАС': 'GUID_FIAS',
    'Код ОКТМО': 'OKTMO_CODE',
    'Способ управления': 'MANAGEMENT_METHOD',
    'ОГРН организации, осуществляющей управление домом': 'MANAGEMENT_OGRN',
    'КПП организации, осуществляющей управление домом': 'MANAGEMENT_KPP',
    'Наименование организации, осуществляющей управление домом': 'MANAGEMENT_NAME',
    'Тип дома': 'HOUSE_TYPE',
    'Состояние': 'CONDITION',
    'Общая площадь дома': 'TOTAL_AREA',
    'Жилая площадь в доме': 'LIVING_AREA',
    'Дата сноса объекта': 'DEMOLITION_DATE',
    'Кадастровый номер': 'CADASTRAL_NUM',
    'Глобальный уникальный идентификатор дома': 'GUID_HOUSE'
    }

In [4]:
houses_data_files = prepare_data.get_filenames_from_dir(
    DATA_FOLDER,
    mandatory_substr='csv')
display(houses_data_files)

['/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_2.csv',
 '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_3.csv',
 '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_1.csv',
 '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_4.csv',
 '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_5.csv',
 '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_7.csv',
 '/Users/affernus/PROJECTS_DATA/hacks/postomat_optim

In [5]:
houses_data_list = []
for hd in houses_data_files:
    houses_data_list.append(pd.read_csv(hd, delimiter=';', low_memory=False))
houses_data = pd.concat(houses_data_list, ignore_index=True).drop_duplicates().reset_index(drop=True)

In [6]:
houses_data['Глобальный уникальный идентификатор дома по ФИАС'] = houses_data['Глобальный уникальный идентификатор дома по ФИАС'].str.upper()

In [7]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    display(houses_data.head(5))
    print(houses_data.shape)

,Адрес ОЖФ,Идентификационный код адреса дома в ГИС ЖКХ,Глобальный уникальный идентификатор дома по ФИАС,Код ОКТМО,Способ управления,"ОГРН организации, осуществляющей управление домом","КПП организации, осуществляющей управление домом","Наименование организации, осуществляющей управление домом",Тип дома,Состояние,Общая площадь дома,Жилая площадь в доме,Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Тип общежития,Тип помещения (блока),Дата сноса объекта,Номер помещения (блока),Номер комнаты,Кадастровый номер,Глобальный уникальный идентификатор дома,Глобальный уникальный идентификатор помещения,Глобальный уникальный идентификатор комнаты
0,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 3",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,396.2,NaN,-,-,-,NaN,NaN,NaN,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09,NaN,NaN
1,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 3А",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,124.9,NaN,-,-,-,NaN,NaN,NaN,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037,NaN,NaN
2,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 4",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,209.3,NaN,-,-,-,NaN,NaN,NaN,NaN,77:18:0170406:50,66eabcec-e02b-408e-9629-d17935b14399,NaN,NaN
3,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 4",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,209.3,NaN,-,-,-,NaN,NaN,NaN,1,NaN,66eabcec-e02b-408e-9629-d17935b14399,NaN,a18b5ed7-98d0-4364-960f-63720f8fee31
4,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 6",855583f2-ce42-4ad0-9c9a-cfca34288c7f,10860E59-E7D1-43C2-A19B-1856644E9B3C,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,145.6,NaN,-,-,-,NaN,NaN,NaN,NaN,77:18:0170406:388,505249be-5ff4-4f7f-bfdc-222aab4cae19,NaN,NaN


(5161209, 23)


In [8]:
not_room = houses_data['Номер помещения (блока)'].isna() & houses_data['Номер комнаты'].isna()
houses_data_not_room = houses_data[not_room].dropna(axis=1, how='all')
houses_data_not_room = houses_data_not_room.drop_duplicates()
print(houses_data_not_room.shape)

(68318, 18)


In [9]:
display(houses_data_not_room.head())

,Адрес ОЖФ,Идентификационный код адреса дома в ГИС ЖКХ,Глобальный уникальный идентификатор дома по ФИАС,Код ОКТМО,Способ управления,"ОГРН организации, осуществляющей управление домом","КПП организации, осуществляющей управление домом","Наименование организации, осуществляющей управление домом",Тип дома,Состояние,Общая площадь дома,Жилая площадь в доме,Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Тип общежития,Дата сноса объекта,Кадастровый номер,Глобальный уникальный идентификатор дома
0,"108809, Москва г, п. Марушкинское, д. Марушкин...",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,396.2,NaN,-,-,-,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09
1,"108809, Москва г, п. Марушкинское, д. Марушкин...",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,124.9,NaN,-,-,-,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037
2,"108809, Москва г, п. Марушкинское, д. Марушкин...",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,209.3,NaN,-,-,-,NaN,77:18:0170406:50,66eabcec-e02b-408e-9629-d17935b14399
4,"108809, Москва г, п. Марушкинское, д. Марушкин...",855583f2-ce42-4ad0-9c9a-cfca34288c7f,10860E59-E7D1-43C2-A19B-1856644E9B3C,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,145.6,NaN,-,-,-,NaN,77:18:0170406:388,505249be-5ff4-4f7f-bfdc-222aab4cae19
5,"108809, Москва г, п. Марушкинское, д. Марушкин...",75f9b36c-5b8e-4c81-9215-b5fc4d462c37,DBC3B094-D0E1-4FC3-A5F7-044B88FC8124,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,224.1,NaN,-,-,-,NaN,50:26:0170402:2279,dae214a7-d984-4c71-9c97-48215a5fa8bf


In [10]:
houses_data['Глобальный уникальный идентификатор дома'].unique().shape, houses_data_not_room['Глобальный уникальный идентификатор дома'].unique().shape

((68159,), (68159,))

In [11]:
prepared_gis_house_data = houses_data_not_room[COLS_MAP.keys()].copy()
prepared_gis_house_data.columns = [
    COLS_MAP[col] for col in prepared_gis_house_data.columns
    ]

In [12]:
prepared_gis_house_data['LIVING_AREA'] = prepared_gis_house_data['LIVING_AREA'].str.replace(',', '.').astype(float)

In [13]:
prepared_gis_house_data[prepared_gis_house_data['GUID_FIAS'].duplicated()]

,ADDRESS_GIS,ADDRESS_GIS_CODE,GUID_FIAS,OKTMO_CODE,MANAGEMENT_METHOD,MANAGEMENT_OGRN,MANAGEMENT_KPP,MANAGEMENT_NAME,HOUSE_TYPE,CONDITION,TOTAL_AREA,LIVING_AREA,DEMOLITION_DATE,CADASTRAL_NUM,GUID_HOUSE
8850,"108811, Москва г, п. Московский, г. Московский...",038dbdfd-cf14-45e3-a635-a4b72bd7e8c9,179457AB-C968-407E-AA18-457B0FDE3302,45952000,УО,1.145044e+12,775101001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""УПРА...",Многоквартирный,Исправный,7010.4,6919.0,NaN,нет,ed2c3ca0-803f-4468-a2f8-bcd48806a1ee
9883,"108811, Москва г, п. Московский, г. Московский...",86f098df-6e53-4f5a-854b-ef623e750b4d,46E720A7-838D-4215-A871-2D8DEC07B492,45952000,УО,1.145044e+12,775101001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""УПРА...",Многоквартирный,Исправный,4671.9,3567.1,NaN,NaN,e497450a-64dc-4ade-b9a7-5a95d5f545bf
98059,"108818, Москва г, п. Десеновское, д. Десна, мк...",79b8e8be-003f-4c01-880c-8fe4e87bb569,BAB778AB-C4CF-4789-A2B3-B434FAC3A860,45944000,УО,1.077746e+12,770601001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЦЕНТ...",Многоквартирный,Исправный,632.1,409.7,NaN,50:21:0140106:1333,68d83c0f-6cdd-4067-8ed1-86e93cea58b3
98093,"108818, Москва г, п. Десеновское, д. Десна, мк...",e6fdfc88-15ec-49e8-b88a-676134f1a62a,E02B340B-2C04-437D-9D14-498C02ECA886,45944000,УО,1.077746e+12,775145001.0,Обособленное подразделение Ватутинки Общества ...,Многоквартирный,Исправный,1440.2,1061.6,NaN,50:21:0140116:670,56785e29-f511-4b08-afe1-e2201915cd2c
98710,"108818, Москва г, п. Десеновское, д. Яковлево,...",a6715a0a-f509-4f22-bf33-2d85abf53bf2,1785C787-01E2-4FFA-85F1-662DE4C3C6B8,45944000,УО,1.077746e+12,770601001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЦЕНТ...",Многоквартирный,Исправный,2726.9,1715.9,NaN,50:21:0140202:579,e3c08c9d-b8d4-464f-85ce-d3bbded2184c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074643,"129075, Москва г, ул. 2-я Новоостанкинская, д. 6",a80cad54-8776-4e8a-bed8-d4859f64056b,FD11D3BB-6F75-4387-8DD1-D709E52EA7F6,45358000,ТСЖ,1.217701e+12,771701001.0,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ НЕДВИЖИМОСТИ ""КОРАЛ...",Многоквартирный,Исправный,9164.1,7434.7,NaN,77:02:0022006:1058,73d4191b-1a67-4f8e-9c1f-a34c7e4be390
5113751,"129128, Москва г, ул. Будайская, д. 5",9d9bd807-ed53-49ee-a8bf-ff7dcb9c6874,9D9BD807-ED53-49EE-A8BF-FF7DCB9C6874,45360000,ТСЖ,1.037716e+12,771601001.0,"ЖИЛИЩНО-СТРОИТЕЛЬНЫЙ КООПЕРАТИВ ""ГОЛУБОЙ ЭКРАН""",Многоквартирный,Исправный,4245.4,2609.2,NaN,77:02:0019003:1000,34f90287-5d60-47a6-8c05-867b327e4761
5125182,"129164, Москва г, ул. Маломосковская, д. 4",06938f00-b3e7-4e92-a30c-859a6a14e720,0C99FC05-89A5-461F-932B-5649F1303523,45349000,УО,1.195081e+12,771701001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВРЕМ...",Многоквартирный,Исправный,22255.0,13994.0,NaN,77:02:0023012:4405,420b9d9c-2d7d-4059-8a88-509db447d4d0
5125183,"129164, Москва г, ул. Маломосковская, д. 4",06938f00-b3e7-4e92-a30c-859a6a14e720,0C99FC05-89A5-461F-932B-5649F1303523,45349000,ТСЖ,1.157746e+12,771701001.0,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ ЖИЛЬЯ ""ЖИЛОЙ КОМПЛЕ...",Многоквартирный,Исправный,22255.0,13994.0,NaN,77:02:0023012:4405,420b9d9c-2d7d-4059-8a88-509db447d4d0


In [14]:
prepared_gis_house_data['OKTMO_CODE'] = prepare_data.codes_to_str(prepared_gis_house_data['OKTMO_CODE'])
prepared_gis_house_data['MANAGEMENT_OGRN'] = prepare_data.codes_to_str(prepared_gis_house_data['MANAGEMENT_OGRN'])
prepared_gis_house_data['MANAGEMENT_KPP'] = prepare_data.codes_to_str(prepared_gis_house_data['MANAGEMENT_KPP'])

In [15]:
prepared_gis_house_data


,ADDRESS_GIS,ADDRESS_GIS_CODE,GUID_FIAS,OKTMO_CODE,MANAGEMENT_METHOD,MANAGEMENT_OGRN,MANAGEMENT_KPP,MANAGEMENT_NAME,HOUSE_TYPE,CONDITION,TOTAL_AREA,LIVING_AREA,DEMOLITION_DATE,CADASTRAL_NUM,GUID_HOUSE
0,"108809, Москва г, п. Марушкинское, д. Марушкин...",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,,,NaN,Жилой,Исправный,396.2,NaN,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09
1,"108809, Москва г, п. Марушкинское, д. Марушкин...",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,,,NaN,Жилой,Исправный,124.9,NaN,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037
2,"108809, Москва г, п. Марушкинское, д. Марушкин...",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,,,NaN,Жилой,Исправный,209.3,NaN,NaN,77:18:0170406:50,66eabcec-e02b-408e-9629-d17935b14399
4,"108809, Москва г, п. Марушкинское, д. Марушкин...",855583f2-ce42-4ad0-9c9a-cfca34288c7f,10860E59-E7D1-43C2-A19B-1856644E9B3C,45949000,Не выбран,,,NaN,Жилой,Исправный,145.6,NaN,NaN,77:18:0170406:388,505249be-5ff4-4f7f-bfdc-222aab4cae19
5,"108809, Москва г, п. Марушкинское, д. Марушкин...",75f9b36c-5b8e-4c81-9215-b5fc4d462c37,DBC3B094-D0E1-4FC3-A5F7-044B88FC8124,45949000,Не выбран,,,NaN,Жилой,Исправный,224.1,NaN,NaN,50:26:0170402:2279,dae214a7-d984-4c71-9c97-48215a5fa8bf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5160120,"129323, Москва г, проезд. Лазоревый, д. 6",689fd70f-3055-4465-8190-21ccac493d57,689FD70F-3055-4465-8190-21CCAC493D57,45361000,УО,1087746440396,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,6892.7,6892.0,NaN,77:02:0014011:1055,9d85374a-7c36-4f60-8cdb-43edc2300929
5160265,"129323, Москва г, проезд. Лазоревый, д. 8",ca25b32a-eb6b-4d25-9dc9-de2d32e321f7,CA25B32A-EB6B-4D25-9DC9-DE2D32E321F7,45361000,УО,1087746440396,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,6892.9,6892.9,NaN,77:02:0014011:1056,d9596c10-41ba-4fb5-9da6-5fe8b03a8fb0
5160410,"129323, Москва г, проезд. Русанова, д. 11",e61605c6-439f-46ac-8feb-fe5faf478145,E61605C6-439F-46AC-8FEB-FE5FAF478145,45361000,УО,5167746441088,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,13456.2,9437.4,NaN,77:02:0014011:13860,a87c74af-1983-4b25-9ae4-529044cf0fb6
5160596,"129323, Москва г, проезд. Русанова, д. 25, кор...",f87cae10-8731-4507-9184-f28ad94ee09a,F87CAE10-8731-4507-9184-F28AD94EE09A,45361000,УО,1087746440396,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,29049.2,21213.0,NaN,77:02:0014011:13149,21357c11-18f4-4487-8281-4324d6df4ba9


In [16]:
prepared_gis_house_data[OBJECT_ID_COL] = prepare_data.get_row_hashes(prepared_gis_house_data)

In [18]:
assert prepared_gis_house_data[OBJECT_ID_COL].duplicated().sum() == 0

In [17]:
prepared_gis_house_data.to_pickle(result_filepath, protocol=4)